In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

pd.options.display.max_rows = 11

# Chap5. Getting Started with Pandas
online help: http://pda.readthedocs.io/en/latest/chp5.html

## Introduction to pandas Data Stuctures
最基本的数据结构是Series和DataFrame，其他还包括Panel, Index, DatetimeIndex, HDFStore  
Series可以理解为一维数组，和arrays或者lists的结构相似  
DataFrame可以理解为二维数组，由一系列的Series构成。每一个Series内的数据，理论为同一类型（但其实在输入的时候没有判断）  
Index，索引，用来搜寻位置

### Series
Series是一种类似于一维数组的对象。它由一组数据（各种NumPy数据类型）以及一组与之相关的数据标签（索引，index）。  
如果没有给数据制定索引，会设定 0 到 N-1 的默认索引。

#### 建立
- 通过list直接建立
- 通过dict直接建立，key默认为index。制定的index overrides keys。


In [2]:
# 通过list建立
# 可以附加index
s1 = Series([4, 7, -5, 3])
s1 = Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])

In [3]:
# 通过dict建立，key默认为index
# 如果制定了index, 则只保留overlap的部分，dict的key中没有的赋值NaN
sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
s3 = Series(sdata)
states = ['California', 'Texas', 'Ohio', 'Oregon']
# 注意Series的index的顺序也会根据改变
s4 = Series(sdata, states)
s4

California        NaN
Texas         71000.0
Ohio          35000.0
Oregon        16000.0
dtype: float64

### 关于索引
- `series.index`
- `series.values`，返回`numpy.ndarray` type
- 对象本身和他的索引都有一个`name`属性，`Series.name`, `Series.index.name`
- 索引可以通过赋值直接更改，但需要全部重新赋值，不能单个更改；DataFrame亦然

In [4]:
# 对象本身和它的索引都有一个name属性
s4.name = 'population'
s4.index.name = 'state'
s4

state
California        NaN
Texas         71000.0
Ohio          35000.0
Oregon        16000.0
Name: population, dtype: float64

### 引用
- 通过位置`series[0]`
- 通过名称`series['a']`, `series.a`

### DataFrame
A DataFrame represents a tabular, spreadsheet-like data structure containing an ordered collection of columns, each of which can be a different value type (numeric, string, boolena, etc.).  

#### 建立
- 通过dict建立，key对应columns
- 嵌套dict，外层key对应columns，内层key对应index
> 1. dict的`values`必须统一长度
> 1. `index`的键入必须和dict的长度一样

In [5]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
       'year': [2000, 2001, 2002, 2001, 2002],
       'pop': [1.5, 1.7, 3.6, 2.4, 2.9]}
df1 = DataFrame(data)

In [6]:
# 双层字典，外层key对应columns，内层key对应index
pop = {'Nevada': {2001:2.4, 2002:2.9},
      'Ohio': {2000:1.5, 2001:1.7, 2002:3.6}}
df2 = DataFrame(pop)
df2

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


#### 引用
- 列
    - `df['col_name']`
    - `df.col_name`
- 行
    - `df.loc[index value]`
    - `df.iloc[index position]`
    - `df[i1:i2]`
        > 可以用`df[i1:i2]`形式的slicing，但不能直接`df[i]`
- m行n列
    - 层级形式`df['col_name']['row_name']`
    > 相当于`df['col_name']`取出了一个Series，然后在对Series进行引用

##### 注意
引用得到的都是reference，不能其中的元素直接进行修改
> `b = df['year']`  
> `b[0] = 1`这样的形式不可以
    

#### 赋值???
- 列的赋值  
    `df['col_name'] = obj`  
    如果多列的话必须长度一致
- 行的赋值  
    `df.iloc[index] = obj`  
    obj是Series的形式，index对应到df中的columns

#### 删除
- 列
    - `del df['col_name']`原地删除
    - `df.drop(cols, axis=1)`创建新的df
- 行
    - `df.drop(index_num/index_name)`

### Index, 索引
- 索引对象是不可变的(immutable)

Pandas中主要的Index对象

Class              | Description
:---               | :---
Index              | 最general的Index对象，将轴标签表示为一个由Python对象组成的NumPy数组
Int64Index         | 针对整数的特殊Index
MultiIndex         | 层次化所以对象，可以看做由元组组成的数组
DatetimeIndex      | 存储纳秒级时间戳（用NumPy的datetime64类型表示）
PeriodIndex        | 针对Period数据（时间间隔）的特殊Index

?? date?

Index的方法和属性

Method             | Description
:---               | :---
append             | 连接另一个Index对象，产生一个新的Index
diff               | 计算差集，得到新的Index
intersection       | 计算交集
union              | 计算并集
isin               | 是否包含在参数集合中的布尔数组
delete             | 删除索引i处的元素，得到新Index
drop               | 删除传入的值，得到新Index
insert             | 将元素插入到索引i处，得到新Index
is_monotonic       | 判断排序，返回boolean
is_unique          | 是否有重复值，返回boolean
unique             | 计算Index中唯一值，返回数组

## Essential Functionality
### 重新索引
`reindex`并不对原df进行操作，而是生成一个新的
- 对值进行内插或填充 `ffill`, `bfill`, `fill_value`
- 可以一次对行与列同时进行重新索引，但差值只在行进行
- 使用带标签索引的ix可以把重新索引做的更简单(? deprecated?)

In [7]:
# simple reindex
s1 = Series([4.5, 7.2, -5.3, 3.6], index=['d','b','a','c'])
s2 = s1.reindex(['a','b','c','c','e'])
s2

a   -5.3
b    7.2
c    3.6
c    3.6
e    NaN
dtype: float64

In [8]:
# fill method
s3 = Series(['blue','purple','yellow'], index=[0,2,4])
s3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

In [9]:
# 列的重新索引
df = DataFrame(np.arange(9).reshape((3,3)), index=['a','c','d'], columns=['Ohio','Texas','California'])
states = ['Texas', 'Utah', 'California']
df.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


In [ ]:
# 可以一次对行与列同时进行重新索引，但差值只在行进行
df.reindex(index=['a','b','c','d'], method='ffill', columns=states)
# error when running

### 索引、选取和过滤 Indexing, Selection, and Filtering
ix, icol, irow都被淘汰了，读documentation
# todo

### 算术运算和数据对齐 Arithmetic and Data Alignment
算术方法, `fill_value` as parameter
- add()
- sub()
- div()
- mul()

### 函数应用和映射 Function Application and Mapping
- NumPy的`ufuncs`也可用于操作pandas对象
    > `np.abs(df)`
- 将函数应用到各列或行所形成的一维数组上
    ```python
    f = lambda x: x.max() - x.min()
    df.apply(f)
    df.apply(f, axis=1)
    ```
- 元素级别的使用`applymap()`，Series中使用`map()`
    ```python
    format = lambda x: '%.2f' % x
    df.applymap(format)
    s.map(format)
    ```

### Sorting and Ranking
#### Sorting
`sort_index()`根据任意一个轴上的索引进行排序，返回新对象
```python
s.sort_index()
df.sort_index(axis=1)
df.sort_index(axis=0, ascending=False)
df.sort_index(by=['col1','col2'])
```

#### Ranking
`rank`根排序关系密切，且会增设一个排名值

### Axis Indexes with Duplicate Values
`df.index.is_unique`

## Summarizing and Computing Descriptive Statistics
- 直接统计 `df.mean()`
- 间接统计 `df.idxmax()`
- 累积型 `df.cumsum()`
- 全面统计 `df.describe()`

描述和汇总统计
Functions         | Descriptions
:---              | :---
count             | 
describe          |
min               |
max               |
argmin, argmax    | 最小值和最大值的索引位置
idxmin, idxmat    | 最小值和最大值的索引值
quantile          | 计算样本的分位数（0到1）
sum               |
mean              |
median            |
mad               | 根据平均值计算平均绝对离差
var               |
std               |
skew              |
kurt              |
cumsum            | 累积和
cummin, cummax    | 累积最大值和累积最小值
cumprod           | 累积积
diff              | 计算一阶差分
pct_change        | 计算百分数变化

### Correlation and Covariance
- Series的`corr`, `cov`用于计算两个Series钟重叠的、非NA的，**按索引对齐**的值的相关关系
- DataFrame的`corr`, `cov`将以DataFrame的形式返回完整的相关系数或协方差矩阵
- DataFrame的`corrwith()`
    - 传入Series计算df中每一列和Series的correlation
    - 传入DataFrame计算对应每一列之间的correlation

In [17]:
from pandas_datareader import data as web

In [ ]:
all_data = {}
for ticker in ['AAPL', 'IBM', 'MSFT']:
    all_data[ticker] = web.DataReader(ticker, 'yahoo', '1/1/2000', '1/1/2001')

In [28]:
price = DataFrame({tic: data['Adj Close'] for tic, data in all_data.items()})
volume = DataFrame({tic: data['Volume'] for tic, data in all_data.items()})

In [35]:
returns = price.pct_change()

In [38]:
returns.tail()

,AAPL,IBM,MSFT
Date,,,
2000-12-22,0.066667,0.091187,0.069064
2000-12-26,-0.020833,-0.047050,0.009421
2000-12-27,0.008510,-0.001475,-0.009333
2000-12-28,0.000000,0.006643,-0.040377
2000-12-29,0.004219,-0.002932,-0.026648


In [39]:
# correlation between series
returns.MSFT.corr(returns.IBM)

0.19321733436645691

In [40]:
# covariance between series
returns.MSFT.cov(returns.IBM)

0.00020701914349988666

In [43]:
# correlation for dataframe
returns.corr()

,AAPL,IBM,MSFT
AAPL,1.000000,0.269484,0.275229
IBM,0.269484,1.000000,0.193217
MSFT,0.275229,0.193217,1.000000


### Unique Values, Value Counts, and Membership
- `serires.unique()` 返回arrays
- `obj.value_counts()` 返回一个Series
- `obj.isin()`

In [52]:
# 频率柱状图
data = DataFrame({'Qu1': [1, 3, 4, 3, 4],
                  'Qu2': [2, 3, 1, 2, 3],
                  'Qu3': [1, 5, 2, 4, 4]})
result = data.apply(pd.value_counts).fillna(0)
result

,Qu1,Qu2,Qu3
1,1.0,1.0,1.0
2,0.0,2.0,1.0
3,2.0,2.0,0.0
4,2.0,0.0,2.0
5,0.0,0.0,1.0


## Handling Missing Data
NA处理方法

Functions | Description
:---      | :---
dropna    | 可调节对缺失值的容忍度，默认drop掉任何包含NA的行/列, `how='all'`值drop全为NA
fillna    | 
isnull    |
notnull   |

### Filtering Out Missing Data
### Filling in Missing Data
```python
df.dropna(0)
df.fillna({'col1': 0.5, 'col2': 1})
df.fillna(0, inplace=True)  # 原地修改
```

## 层次化索引 Hierachical Indexing
对于一个DataFrame，每条轴都可以有分层索引

##### 引用
```python
df['lvl1', 'lvl2']
```


In [62]:
# Series建立，naive法
data = Series(np.random.randn(10), index=[['a', 'a', 'a', 'b', 'b', 'b' ,'c', 'c', 'd', 'd'],
                                         [1, 2, 3, 1, 2, 3, 1, 2, 2, 3]])
data

a  1    1.114437
   2   -0.812213
   3   -0.588589
b  1   -0.965205
   2    0.895828
   3    1.147688
c  1   -0.402138
   2   -0.300804
d  2    1.331312
   3    0.006706
dtype: float64

In [64]:
# 引用
data['b', 2]
data['b']

0.89582841695585658

In [70]:
# DataFrame建立，naive法
frame = DataFrame(np.arange(12).reshape((4, 3)),
                 index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                 columns=[['Ohio', 'Ohio', 'Colorado'],
                         ['Green', 'Red', 'Green']])
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [69]:
# 创建 MultiIndex
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                      names=['state', 'color'])

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           labels=[[1, 1, 0], [0, 1, 0]],
           names=['state', 'color'])

### Reordering and Sorting Levels



In [71]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [79]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [80]:
frame.swaplevel('state', 'color', axis=1).sort_index(level=0, axis=1)

color        Green       Red
state     Colorado Ohio Ohio
key1 key2                   
a    1           2    0    1
     2           5    3    4
b    1           8    6    7
     2          11    9   10

In [74]:
frame.swaplevel?

### Summary Statistics by Level


In [81]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [82]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Using a DataFrame's Columns
- `df.set_index()`
- `df.reset_index()`

## Other pandas Topics
### Integer Indexing
```python
s = Series(np.arange(3))
s[-1]  # error
```
这里我们有一个含有0、1、2的索引，但是很难判断出用户想要什么（基于标签或位置的索引）。如果轴索引还有索引器，那么根据整数进行数据选取的操作将总是面向标签的。

如果需要可靠的、不考虑索引类型的、基于位置的索引，可以使用`iloc[]`


In [88]:
# 没有标明索引的series生成的整数索引
s = Series(np.arange(3))
s.index

RangeIndex(start=0, stop=3, step=1)

In [92]:
# 会发现标明了索引，即便是整数型的，其实和没有标明而生成的索引是不同类型的
s = Series(np.arange(3), index=[1, 2, 3])
s.index
# s[-1] 仍然报错

Int64Index([1, 2, 3], dtype='int64')

In [96]:
s = Series(np.arange(3), index=['a', 'b', 'c'])
s[-1]

2

### Panel Data
